In [3]:
from google.colab import files
from keras import utils
import os
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Conv2D

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')
# http://marsyas.info/downloads/datasets.html

Mounted at /content/drive


In [31]:
import zipfile
import io
z = zipfile.ZipFile('/content/drive/MyDrive/Best of France.zip', 'r')
z.extractall()
genres = ''.split()





In [57]:
# подготовка данных признаков
def get_features(y, sr):

  chroma_stft = np.mean(librosa.feature.chroma_stft(y = y, sr = sr))
  rmse = np.mean(librosa.feature.rms( y = y))
  spec_cent = np.mean(librosa.feature.cpectral_centroid(y = y, sr = sr))
  spec_bw = np.mean(librosa.feature.cpectral_bandwidth(y = y, sr = sr))
  rolloff = np.mean(librosa.feature.cpectral_rilloff(y = y, sr = sr))
  zcr = np.mean(librosa.feature.zero_crossing_rate(y))
  mfcc = librosa.feature.mfcc(y = y, sr = sr)

  out = []
  out.append(chroma_stft)
  out.append(rmse)
  out.append(spec_cent)
  out.append(spec_bw)
  out.append(rolloff)
  out.append(zcr)
  out.append(mfcc)

  for e in mfcc:
    out.append(np.mean(e))

  return out



In [58]:
# подготовка выборки
X_train = []
Y_train = []

for i in range(len(genres)):
  g = genres[i]
  for filename in os.listdir(f'./genres/{g}'):
    songname = f'./genres/{g}/{filename}'
    y, sr = librosa.load(songname, mono=True, duration = 30)
    out = get_features(y, sr)

    X_train.append(out)
    Y_train.append(utils.to_categorical(i, len(genres)))
print("Жанр", g, "готов")

X_train = np.array(X_train)
Y_train = np.array(Y_train)


Жанр Paul готов


In [59]:
# копии
X_train_backup = X_train.copy()
Y_train_backup = Y_train.copy()

In [60]:
# распределение классов
y_train_class = np.argmax(Y_train, axis = 1)
print(y_train_class)

AxisError: ignored

In [61]:
# сравнение размерности
print(X_train.shape)
print(Y_train.shape)
print(y_train_class.shape)

(0,)
(0,)


NameError: ignored

In [62]:
# стандартизация
scaler = StandardScaler
X_train - scaler.fit_transform(np.array(X_train, dtype = float))

TypeError: ignored

In [63]:
# разделение выборки
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train_class, test_size = 0.1)

NameError: ignored

In [64]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(0,)


NameError: ignored

In [71]:
# нейронная сеть
indexes = range(0)

model = Sequential()
model.add(Dense(256, activation = 'elu', input_shape =(len(indexes),)))
model.add(Dense(128, activation = 'elu'))
model.add(Dense(64, activation = 'elu'))
model.add(Dense(32, activation = 'elu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(optimizer=Adam(lr=1e-4),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
history = model.fit(X_train,
                    y_train,
                    epochs = 100,
                    batch_size = 20,
                    validation_data=(X_test, y_test))

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.show()




NameError: ignored